In [1]:
import os
import openai
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import make_pipeline
from sklearn.metrics import classification_report
from joblib import dump
from tqdm import tqdm


In [4]:
API_KEY = "sk-svcacct-XoDm82FIvbsPpvMa1cHNvQ6jHvCKZ-x6pERpSKy68fspNGVPewiDjtSZkC1ge75gXalT3BlbkFJhmLZ82yo42HAo2sP3mkiwV8crramxS7FkX63xI1w7PvBehiBTphZGHD7xHOVMveVJEgA"

client = openai.OpenAI(api_key=API_KEY)

# -------- Config --------
EMBEDDING_MODEL = "text-embedding-3-small"
DATA_PATH = "sofmattress_train.csv"  
MODEL_PATH = "model.joblib"
ENCODER_PATH = "label_encoder.joblib"

# -------- Helper to get embeddings --------
def get_embedding(text: str, model=EMBEDDING_MODEL) -> list:
    response = client.embeddings.create(
        input=[text],
        model=model
    )
    return response.data[0].embedding

# -------- Load and process dataset --------
df = pd.read_csv(DATA_PATH)

# Encode labels
label_encoder = LabelEncoder()
df['label_encoded'] = label_encoder.fit_transform(df['label'])

# Get embeddings
print("Generating embeddings...")
embeddings = []
for text in tqdm(df["sentence"].tolist()):
    embeddings.append(get_embedding(text))

# Train model
print("Training model...")
clf = LogisticRegression(max_iter=1000, class_weight="balanced")
clf.fit(embeddings, df["label_encoded"])

# Save model and label encoder
dump(clf, MODEL_PATH)
dump(label_encoder, ENCODER_PATH)

print("✅ Model and encoder saved!")

Generating embeddings...


100%|██████████| 328/328 [03:12<00:00,  1.71it/s]

Training model...
✅ Model and encoder saved!


In [5]:
import joblib
clf = joblib.load("model.joblib")
label_encoder = joblib.load("label_encoder.joblib")


In [6]:
import numpy
print(numpy.__version__)

2.2.5
